In [1]:
# Use networkx library to represent graphs efficiently & numpy for matrix operations & time for system time
import networkx as nx
import numpy as np
import time as t
import scipy as sp
from scipy.sparse import csr_matrix
from scipy.sparse import dok_matrix

In [2]:
# Function to return the PageRank of all nodes in a given webgraph G using the basic iterative method
def PageRankIterative(G, damping_factor = 0.85, error_tolerance = 0.1, max_iterations = 100):
# Initialise number of iterations of algorithm to 0
    number_iterations = 0
# Initialise PageRank of all nodes to be initially the same value i.e 1/number_of_nodes
    input_ranks = []
    number_of_nodes = G.number_of_nodes()
    for i in range(0,number_of_nodes):
        input_ranks.append(1/number_of_nodes)
# Iterations of PageRank Algorithm
    while number_iterations <= max_iterations:
        number_iterations = number_iterations + 1
        output_ranks = []
# Initialise PageRanks of next iteration to 0 for all nodes
        for i in range(0,number_of_nodes):
            output_ranks.append(0) 
# Compute PageRank iteratively node by node
        for i in range(0,number_of_nodes):
            number_out_edges = G.out_degree(i+1)
# Take care of dangling nodes by distributing its PageRank evenly to all nodes(randomly move to any node)
            if number_out_edges == 0:
                for j in range(0,number_of_nodes):
                    output_ranks[j] = output_ranks[j] + (input_ranks[i]/number_of_nodes)
# Find list of nodes which have a link to the current node
            number_in_edges = G.in_degree(i+1)
            predecessors = list(G.predecessors(i+1))
# Increment PageRank of current node to sum of PageRanks of all nodes linked to current node divided by number of outward links for each of those nodes
            for j in range(0,number_in_edges):
                output_ranks[i] = output_ranks[i] + (input_ranks[predecessors[j]-1]/G.out_degree(predecessors[j]))
# Introduce the damping factor to increase convergence rate and ensure sum of all PageRanks is 1 at all iterations
        for i in range(0,number_of_nodes):
            output_ranks[i] = (damping_factor * output_ranks[i]) + ((1-damping_factor)/number_of_nodes)
# Calculate difference between PageRanks of current and next iteration to check for convergence
        difference = 0.0
        for i in range(0,number_of_nodes):
            difference = difference + abs(output_ranks[i] - input_ranks[i])
        #print(difference)
# Update PageRanks
        input_ranks = output_ranks
# Check for convergence against a tolerance value
        if(difference < error_tolerance):
            #print(number_iterations)
            break
# Return PageRanks of all the nodes
    return output_ranks

In [3]:
# Function to return the PageRank of all nodes in a given webgraph G using the matrix power method
def PageRankPowerMethod(G, damping_factor = 0.85, error_tolerance = 0.1, max_iterations = 100):
# Initialise number of iterations of algorithm to 0 and initialise a square matrix of size (number_of_nodes * number_of_nodes)
    number_iterations = 0
    number_of_nodes = G.number_of_nodes()
    GoogleMatrix = np.zeros((number_of_nodes,number_of_nodes))
# Initialise PageRank of all nodes to be initially the same value i.e 1/number_of_nodes
    InputRanks = np.full((number_of_nodes,1),1/number_of_nodes)
# Procedure to fill in values of matrix GoogleMatrix:
    # GoogleMatrix[i][j] = 0 if no link exists from node j to node i
    # GoogleMatrix[i][j] = 1/number of links from node j if link exists from node j to node i
    # In case of a dangling node j, randomly move to any of the nodes i.e GoogleMatrix[i][j] = 1/number_of_nodes for all i
    for i in range(0,number_of_nodes):
        number_out_edges = G.out_degree(i+1)
        if number_out_edges == 0:
            for j in range(0,number_of_nodes):
                GoogleMatrix[j][i] = 1/number_of_nodes
        else:
            successors = list(G.successors(i+1))
            for j in range(0,number_out_edges):
                GoogleMatrix[successors[j]-1][i] = 1/number_out_edges
# Iterations of PageRank Algorithm
    while number_iterations < max_iterations:
        number_iterations = number_iterations + 1
# Pagerank in next iteration = d(G * R) + (1-d)/N
        OutputRanks = np.matmul(GoogleMatrix,InputRanks) * damping_factor + ((1-damping_factor)/(number_of_nodes))
# Calculate difference between PageRanks of current and next iteration to check for convergence
        difference = sum(sum(abs(OutputRanks - InputRanks)))
        #print(difference)
# Update PageRanks
        InputRanks = OutputRanks
# Check for convergence against a tolerance value
        if(difference < error_tolerance):
            #print(number_iterations)
            break
# Return PageRanks of all the nodes
    return OutputRanks 
    

In [4]:
def PageRankInnerOuterMethod(G, damping_factor_alpha = 0.85, damping_factor_beta = 0.5, error_tolerance_outer = 0.1,error_tolerance_inner = 0.1, max_iterations_outer = 100, max_iterations_inner = 100):
# Inner Outer iteration solves for a smaller damping factor first and then uses that result to solve for the given damping factor
    if (damping_factor_alpha < damping_factor_beta):
        return "Error: Alpha must be greater than Beta"
# Initialise number of iterations of algorithm to 0 and initialise a square matrix of size (number_of_nodes * number_of_nodes)
    number_iterations = 0
    number_of_nodes = G.number_of_nodes()
    GoogleMatrix = np.zeros((number_of_nodes,number_of_nodes))
# Initialise PageRank of all nodes to be initially the same value i.e 1/number_of_nodes
    InputRanks = np.full((number_of_nodes,1),1/number_of_nodes)
# Procedure to fill in values of matrix GoogleMatrix:
# GoogleMatrix[i][j] = 0 if no link exists from node j to node i
# GoogleMatrix[i][j] = 1/number of links from node j if link exists from node j to node i
# In case of a dangling node j, randomly move to any of the nodes i.e GoogleMatrix[i][j] = 1/number_of_nodes for all i
    for i in range(0,number_of_nodes):
        number_out_edges = G.out_degree(i+1)
        if number_out_edges == 0:
            for j in range(0,number_of_nodes):
                GoogleMatrix[j][i] = 1/number_of_nodes
        else:
            successors = list(G.successors(i+1))
            for j in range(0,number_out_edges):
                GoogleMatrix[successors[j]-1][i] = 1/number_out_edges
# Personalisation Vector set to be equal to 1/number_of_nodes for all nodes i.e all nodes are equally likely to be visited in the case of a random choice
    personalisation_vector = np.full((number_of_nodes,1),1/number_of_nodes)
# Initialise PageRanks to be equal to the Personalisation Vector i.e all equal
    InputRanks = personalisation_vector
# y vector used during Inner-Outer Iteration
    y = np.matmul(GoogleMatrix,InputRanks)
# Pagerank in next Outer iteration = alpha(G * R) + (1-alpha)/N
    OutputRanks = (damping_factor_alpha * y + (1-damping_factor_alpha) * personalisation_vector)
# Calculate difference between PageRanks of current and next Outer iteration to check for convergence
    difference_outer = sum(sum(abs(OutputRanks - InputRanks)))
# Repeat Outer iterations until convergence
    while (difference_outer > error_tolerance_outer):
# Calculate f as (alpha-beta)y + (1-alpha)/N
# f used in Inner iterations
        f = (damping_factor_alpha - damping_factor_beta) * y + (1 - damping_factor_alpha) * personalisation_vector
# Set difference_inner to 100 to ensure atleast 1 iteration of inner loop
        difference_inner = 100
# Repeat Inner iterations until convergence 
        while (difference_inner > error_tolerance_inner):
# Update InputRanks and y in each Inner iteration till the InputRanks converge for smaller damping factor beta 
            InputRanks = f + (damping_factor_beta * y)
            y = np.matmul(GoogleMatrix,InputRanks)
# Calculate difference between PageRanks of current and next Inner iteration to check for convergence
            difference_inner = sum(sum(abs(f + (damping_factor_beta * y) - InputRanks)))
# Update OutputRanks in each Outer iteration till the OutputRanks and InputRanks converge for damping factor alpha
        OutputRanks = (damping_factor_alpha * y + (1-damping_factor_alpha) * personalisation_vector)
        difference_outer = sum(sum(abs(OutputRanks - InputRanks)))
# Set final PageRanks of all nodes in OutputRanks
    OutputRanks = (damping_factor_alpha * y + (1-damping_factor_alpha) * personalisation_vector)
# Return PageRanks of all the nodes
    return OutputRanks

In [5]:
# Function to return the PageRank of all nodes in a given webgraph G using the improved matrix power method
def PageRankPowerMethodSparse(G, damping_factor = 0.85, error_tolerance = 0.1, max_iterations = 100):
# Initialise number of iterations of algorithm to 0 and initialise 3 lists for rows,columns and data of the sparse matrix
    number_iterations = 0
    number_of_nodes = G.number_of_nodes()
    rows = []
    columns = []
    data = []
# Initialise PageRank of all nodes to be initially the same value i.e 1/number_of_nodes
    InputRanks = np.full((number_of_nodes,1),1/number_of_nodes)
# Procedure to fill in values of matrix GoogleMatrix:
    # GoogleMatrix[i][j] = 0 if no link exists from node j to node i
    # GoogleMatrix[i][j] = 1/number of links from node j if link exists from node j to node i
    # In case of a dangling node j, randomly move to any of the nodes i.e GoogleMatrix[i][j] = 1/number_of_nodes for all i
    for i in range(0,number_of_nodes):
        number_out_edges = G.out_degree(i+1)
        if number_out_edges == 0:
            for j in range(0,number_of_nodes):
                rows.append(j)
                columns.append(i)
                data.append(1/number_of_nodes)
        else:
            successors = list(G.successors(i+1))
            for j in range(0,number_out_edges):
                rows.append(successors[j]-1)
                columns.append(i)
                data.append(1/number_out_edges)
# Construct GoogleMatrixSparse in Compressed Sparse Matrix(CSR) format using the data,rows and columns lists.
    GoogleMatrixSparse = csr_matrix((data,(rows,columns)), shape=(number_of_nodes,number_of_nodes))
# Iterations of PageRank Algorithm
    while number_iterations < max_iterations:
        number_iterations = number_iterations + 1
# Pagerank in next iteration = d(G * R) + (1-d)/N
        OutputRanks = GoogleMatrixSparse.dot(InputRanks) * damping_factor + ((1-damping_factor)/(number_of_nodes))
# Calculate difference between PageRanks of current and next iteration to check for convergence
        difference = sum(sum(abs(OutputRanks - InputRanks)))
        #print(difference)
# Update PageRanks
        InputRanks = OutputRanks
# Check for convergence against a tolerance value
        if(difference < error_tolerance):
            #print(number_iterations)
            break
# Return PageRanks of all the nodes
    return OutputRanks 
    

In [33]:
# Function to analyze PageRank accuracy and computation time for different damping factor values
def DampingFactorAnalysis():
# Initially set damping factor to 1 and other parameters for the PageRank Algorithm   
    curr_damping_factor = 1
    error_tolerance = 0.001
    max_iterations = 100
    difference = 0
    true_pageranks = 0
    max_time = 0
# Run PageRank Algorithm for different values of damping factor and find which value leads to fastest convergence
    for i in range(0,11):
        curr_damping_factor = 1 - (i/10)
# Load a random graph with 20000 nodes and 1/1000th probability of an edge being added between 2 nodes
        G = nx.fast_gnp_random_graph(20000,1/1000).to_directed()
        G = nx.convert_node_labels_to_integers(G,1)
        number_of_nodes = G.number_of_nodes()
# Measure time taken for the PageRank to be computed using the iterative method
        start_time = t.process_time()
        pageranks = PageRankPowerMethod(G, curr_damping_factor, error_tolerance, max_iterations)
        end_time = t.process_time()
        total_time = end_time - start_time
# Set maximum time to time if i is 0
        if (i == 0):
            max_time = total_time
            true_pageranks = pageranks
        error = 0.0
        for j in range(0,number_of_nodes):
            error = difference + abs(true_pageranks[j] - pageranks[j])
        print("Damping Factor: ",(10-i)/10, "Time Taken: ", total_time, "Error compared to True PageRank: ", error)

In [16]:
# Load a DiGraph where nodes are labelled from 1 to number_of_nodes
def LoadGraph():
# Create an empty directed graph
    G = nx.DiGraph()
# Use the Stanford web graph from 2002 as a large dataset
    dataset = open("../input/webgraphs/web-Stanford.txt","r")
    i = 0
    for line in dataset:
        i = i+1
# Information about graph in first 4 lines
        if i < 5:
            continue
        else:
# Add each edge to the graph
            node1, node2 = (int(val) for val in line.split())
            G.add_edge(node1,node2)
    G = nx.convert_node_labels_to_integers(G,1)
    return G

In [8]:
# Check that graph has successfully created all nodes and edges
G = LoadGraph()
print(G.number_of_nodes())
print(G.number_of_edges())

281903
2312497


In [34]:
# Analyze different damping factors
DampingFactorAnalysis()

Damping Factor:  1.0 Time Taken:  4.700951138000164 Error compared to True PageRank:  [0.]
Damping Factor:  0.9 Time Taken:  3.035951218000264 Error compared to True PageRank:  [1.31736264e-05]
Damping Factor:  0.8 Time Taken:  2.8101013580003382 Error compared to True PageRank:  [3.00174591e-06]
Damping Factor:  0.7 Time Taken:  2.8382574340002975 Error compared to True PageRank:  [8.05481481e-06]
Damping Factor:  0.6 Time Taken:  2.6750187079996977 Error compared to True PageRank:  [2.06254881e-08]
Damping Factor:  0.5 Time Taken:  2.6565456550001727 Error compared to True PageRank:  [7.94722487e-06]
Damping Factor:  0.4 Time Taken:  2.464489378000053 Error compared to True PageRank:  [5.20967198e-06]
Damping Factor:  0.3 Time Taken:  2.5023031830000946 Error compared to True PageRank:  [2.98449149e-06]
Damping Factor:  0.2 Time Taken:  2.5237939290000213 Error compared to True PageRank:  [4.99381086e-06]
Damping Factor:  0.1 Time Taken:  2.340845690000151 Error compared to True Page

In [23]:
damping_factor = 0.85
error_tolerance = 0.1
max_iterations = 100
#pageranks_power = PageRankPowerMethod(G, damping_factor, error_tolerance, max_iterations)
pageranks_iterative = PageRankIterative(G, damping_factor, error_tolerance, max_iterations)
print(pageranks_iterative[0],pageranks_iterative[1],pageranks_iterative[2], sum(pageranks_iterative))
#print(pageranks_power[0],pageranks_power[1],pageranks_power[2],sum(pageranks_power))

5.333815740521636e-07 2.8607493192967646e-06 2.8607493192967646e-06 0.9999999999989433


In [24]:
H = nx.DiGraph()
H.add_edge(1,2)
H.add_edge(2,3)
H.add_edge(3,1)
H.add_edge(1,4)
H.add_edge(2,4)
H.add_edge(3,4)
damping_factor = 1
error_tolerance = 0.1
max_iterations = 100
pageranks_power = PageRankPowerMethod(H, damping_factor, error_tolerance, max_iterations)
pageranks_iterative = PageRankIterative(H, damping_factor, error_tolerance, max_iterations)
pageranks_inner_outer = PageRankInnerOuterMethod(H,damping_factor)
print(pageranks_power)
print(pageranks_iterative)
print(pageranks_inner_outer)


[[0.203125]
 [0.203125]
 [0.203125]
 [0.390625]]
[0.203125, 0.203125, 0.203125, 0.390625]
[[0.203125]
 [0.203125]
 [0.203125]
 [0.390625]]


In [25]:
I = nx.les_miserables_graph().to_directed()
I = nx.convert_node_labels_to_integers(I,1)
damping_factor = 1
error_tolerance = 0.1
max_iterations = 100
pageranks_power = PageRankPowerMethod(I, damping_factor, error_tolerance, max_iterations)
pageranks_iterative = PageRankIterative(I, damping_factor, error_tolerance, max_iterations)
pageranks_inner_outer = PageRankInnerOuterMethod(I,damping_factor)
print(sum(pageranks_power))
print(sum(pageranks_iterative))
print(sum(pageranks_inner_outer))

[1.]
1.0000000000000002
[1.]


In [30]:
I = nx.fast_gnp_random_graph(20000,1/1000).to_directed()
I = nx.convert_node_labels_to_integers(I,1)
print(I.number_of_nodes())
print(I.number_of_edges())
damping_factor = 0.85
error_tolerance = 0.1
max_iterations = 100
start_time = t.process_time()
pageranks = PageRankPowerMethod(I,damping_factor, error_tolerance, max_iterations)
end_time = t.process_time()
time_before_sparse_matrix_vector_multiplication = end_time - start_time
print("PageRanks: ",pageranks)
print("Sum of PageRanks: ",sum(pageranks))
print("Time Taken: ",time_before_sparse_matrix_vector_multiplication)

20000
400052
PageRanks:  [[5.56615540e-05]
 [4.53666352e-05]
 [3.80024050e-05]
 ...
 [4.21529162e-05]
 [4.20769084e-05]
 [3.51532350e-05]]
Sum of PageRanks:  [1.]
Time Taken:  2.290788250999867


In [31]:
print(I.number_of_nodes())
print(I.number_of_edges())
damping_factor = 0.85
error_tolerance = 0.1
max_iterations = 100
start_time = t.process_time()
pageranks = PageRankPowerMethodSparse(I,damping_factor,error_tolerance)
end_time = t.process_time()
time_after_sparse_matrix_vector_multiplication = end_time - start_time
print("PageRanks: ",pageranks)
print("Sum of PageRanks: ",sum(pageranks))
print("Time Taken: ",time_after_sparse_matrix_vector_multiplication)

20000
400052
PageRanks:  [[5.56615540e-05]
 [4.53666352e-05]
 [3.80024050e-05]
 ...
 [4.21529162e-05]
 [4.20769084e-05]
 [3.51532350e-05]]
Sum of PageRanks:  [1.]
Time Taken:  0.7757984439999746
